## Informe de Extracción, Transformación y Carga de Datos: ETL_user_items

En esta sección, nos sumergiremos en el proceso de Extracción, Transformación y Carga (ETL) del set de datos australian_user_items donde se registran los juegos en la plataforma Steam por usuario. El objetivo principal es preparar estos datos para su análisis, asegurándonos de que sean aptos y consistentes.

Comenzaremos importando las bibliotecas esenciales. Asegúrese de tener estas bibliotecas instaladas previamente para garantizar una ejecución sin contratiempos.

A lo largo del informe, nos enfocaremos en abordar problemas potenciales en los datos, aplicar técnicas de limpieza y preprocesamiento, y finalmente, almacenar los datos transformados para futuras exploraciones y análisis.

### Importar Bibliotecas

In [1]:
# Importamos pandas para el análisis de datos tabulares
import pandas as pd
from pandas import json_normalize

# NumPy proporciona soporte para arreglos y matrices multidimensionales
import numpy as np

# El módulo os permite interactuar con el sistema operativo
import os

# Gdown facilita la descarga de archivos desde Google Drive utilizando su ID
import gdown

# JSON es un formato común para el intercambio de datos, y Python tiene soporte incorporado para trabajar con JSON
import json

import warnings
warnings.filterwarnings("ignore")

### 1. Cargar el conjunto de datos original

Fuente de datos: **australian_user_items.json**

Este archivo está disponible en Google Drive con acceso compartido. Puede descargarlo en el siguiente enlace.
Datasets: https://bit.ly/47J98PN

In [10]:
ruta = 'australian_users_items.json'

In [11]:
with open(ruta, 'r', encoding='utf-8') as f:
    data = f.readlines()

# vamos a convertir las lineas en registros
records = [eval(line.strip()) for line in data]

# Creamos el DataFrame a partir de los records o registros
df_UserItems= pd.DataFrame(records)

In [4]:
respaldo = df_UserItems

### 2. Explorar y entender el conjunto de datos

In [12]:
df_UserItems.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [13]:
# Obtener información general del DataFrame
print("\nInformación general del DataFrame:")
df_UserItems.info()


Información general del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


Es importante destacar que no se observan valores nulos en ninguna de las columnas, lo que sugiere que todos los usuarios tienen información completa en estas columnas.

Para obtener una comprensión más profunda de estos datos se estará examinando algunos registros específicos para comprender mejor la naturaleza de los elementos.La columna 'items' es una lista, así que se explora la misma para conocer su estructura.

### 3. Limpiar y preprocesar el conjunto de datos

- Manejar valores nulos o duplicados.
- Identificar y eliminar columnas irrelevantes (si es necesario).
- Normalizar columnas

In [14]:
df_UserItems['items'][0]

[{'item_id': '10',
  'item_name': 'Counter-Strike',
  'playtime_forever': 6,
  'playtime_2weeks': 0},
 {'item_id': '20',
  'item_name': 'Team Fortress Classic',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '30',
  'item_name': 'Day of Defeat',
  'playtime_forever': 7,
  'playtime_2weeks': 0},
 {'item_id': '40',
  'item_name': 'Deathmatch Classic',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '50',
  'item_name': 'Half-Life: Opposing Force',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '60',
  'item_name': 'Ricochet',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '70',
  'item_name': 'Half-Life',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '130',
  'item_name': 'Half-Life: Blue Shift',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '300',
  'item_name': 'Day of Defeat: Source',
  'playtime_forever': 4733,
  'playtime_2weeks': 0},
 {'item_id': '240',
  'item_name': 'Counter-Strike: S

Efectivamente la columna 'items' se encuentra anidada, es decir, es una lista de diccionarios. A continuación, se normaliza la columna de manera de obtener una columna por cada clave de ese diccionario, manteniendo la trazabilidad de las columnas 'steam_id','items_count','user_id' y 'user_url'.

In [15]:
# Normalizar la columna 'items'
df_UserItems = pd.json_normalize(records, record_path=['items'], meta=['steam_id','items_count','user_id','user_url'] )

In [16]:
# Encuentra los registros duplicados
duplicados = df_UserItems[df_UserItems.duplicated(subset=['steam_id','item_id','playtime_forever'])]
duplicados.shape

(59117, 8)

In [17]:
#Borramos dulplicados
df_UserItems = df_UserItems.drop_duplicates(keep='first')

In [18]:
# Convertir la columna 'playtime_forever' de minutos a horas
df_UserItems['hours_game'] = df_UserItems['playtime_forever'] / 60

# Redondear los valores a dos decimales (opcional)
df_UserItems['hours_game'] = df_UserItems['hours_game'].round(2)

In [19]:
# Obtener los 5 valores máximos
top_5_max = df_UserItems.nlargest(5, 'hours_game')

# Obtener los 5 valores mínimos
top_5_min = df_UserItems.nsmallest(5, 'hours_game')

# Mostrar los resultados
print("Top 5 valores máximos:")
print(top_5_max[['hours_game']])

print("\nTop 5 valores mínimos:")
print(top_5_min[['hours_game']])

Top 5 valores máximos:
         hours_game
587715     10712.88
2499068    10588.25
4075729    10540.87
1495340    10223.52
1836985    10001.13

Top 5 valores mínimos:
   hours_game
1         0.0
3         0.0
4         0.0
5         0.0
6         0.0


In [20]:
# Contar las filas con valor 0 en la columna 'hours_game'
filas_con_cero = (df_UserItems['hours_game'] == 0).sum()

# Imprimir el resultado
print(f"Existen {filas_con_cero} filas con valor 0 en la columna 'hours_game'.")

Existen 1847730 filas con valor 0 en la columna 'hours_game'.


In [21]:
# Eliminar las filas con valor 0 en la columna 'hours_game'
df_UserItems = df_UserItems[df_UserItems['hours_game'] != 0]

Se identificaron 1,847,730 registros con un valor de 0 en la columna 'hours_game'. Dado que el análisis de horas jugadas es un requisito, se ha tomado la decisión de eliminar todos los registros con este valor nulo en la columna 'hours_game', ya que no contribuyen al estudio propuesto.

In [22]:
#Eliminamos columna no relevantes
df_UserItems = df_UserItems.drop(['playtime_2weeks','user_url','items_count','item_name','steam_id','playtime_forever'], axis=1)
df_UserItems.columns

Index(['item_id', 'user_id', 'hours_game'], dtype='object')

In [23]:
df_UserItems

,item_id,user_id,hours_game
0,10,76561197970982479,0.10
2,30,76561197970982479,0.12
8,300,76561197970982479,78.88
9,240,76561197970982479,30.88
10,3830,76561197970982479,5.55
...,...,...,...
5153202,304930,76561198329548331,11.28
5153203,227940,76561198329548331,0.72
5153206,388490,76561198329548331,0.05
5153207,521570,76561198329548331,0.07


### 4. Guardar el conjunto de datos limpio
Guardamos el conjunto de datos limpio en un nuevo archivo CSV/JSON/PARQUET para facilitar el acceso y la reutilización.

In [24]:
# Los archivos se almacenan en local 
df_UserItems.to_csv('user_items_cleaned.csv', index=False)
df_UserItems.to_json('user_items_cleaned.json', orient='records', lines=True)
df_UserItems.to_parquet('user_items_cleaned.parquet', index=False)